In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import re  # Importa o módulo de expressões regulares

def extrair_dados_miracles(soup_categoria):
    try:
        data = []
        for row in soup_categoria.find_all('div', class_='table-responsive')[1].find('table').find_all('tr')[1:]:
            columns = row.find_all('td')
            icon = columns[0].find('img')['src']
            name = columns[1].get_text(strip=True)
            description = columns[2].get_text(strip=True)
            fp_cost = columns[3].get_text(strip=True)
            slots_used = columns[4].get_text(strip=True)
            faith_required = columns[5].get_text(strip=True)
            
            # Extrai o texto da coluna Acquired From diretamente das tags <li>
            acquired_from = ' / '.join(re.sub(r'([a-z])([A-Z])', r'\1 \2', li.get_text(strip=True)) for li in columns[6].find_all('li'))

            data.append({
                'Icon': icon,
                'Miracle': name,
                'Description': description,
                'FP cost': fp_cost,
                'Slots used': slots_used,
                'Faith required': faith_required,
                'Acquired From': acquired_from
            })

        miracles_df = pd.DataFrame(data)

        return miracles_df

    except Exception as e:
        print(f"Erro ao extrair dados da categoria Miracles: {e}")
        return None

# URL da categoria "Miracles"
url_miracles = "https://darksouls3.wiki.fextralife.com/Miracles"

# Faz a requisição HTTP para a página da categoria
response_miracles = requests.get(url_miracles)

if response_miracles.status_code == 200:
    html_miracles = response_miracles.text
    soup_miracles = BeautifulSoup(html_miracles, 'html.parser')  # Use o parser padrão aqui

    # Extrai os dados da categoria "Miracles"
    miracles_df = extrair_dados_miracles(soup_miracles)

    if miracles_df is not None and not miracles_df.empty:
        print("Dados da categoria Miracles:")
        # Exibe os dados como tabela
        print(tabulate(miracles_df, headers='keys', tablefmt='pretty', showindex=False))
    else:
        print("Falha ao extrair dados específicos da categoria Miracles ou o DataFrame está vazio.")
else:
    print(f"Falha ao obter a página da categoria. Código de status: {response_miracles.status_code}")
